# Aggregating Task Data Extracted from Job Ads for Applications

## From Tasks to [O-NET Scores](https://www.onetonline.org/help/online/scales)

We go from data:

```

"[('1117', 'Prepare required paperwork pertaining to departmental functions.'), ('16778', 'Confer with departments, such as marketing, business development, or operations, to coordinate product development or improvement.')]"

```

To visualization of Skill Importance by Occupation:

![Visualization](data/demo/TaskMatch/newplot.png)

In [17]:
## Load Packages and Data, and Merge O-NET Crosswalks

In [18]:
import pandas as pd
import ast
import numpy as np
from operator import itemgetter

# Load the Output
data = pd.read_excel("data/demo/CREAM/coded_output.xlsx")

#Load in Files to Merge
dwas = pd.read_excel("data/demo/TaskMatch/Tasks to DWAs.xlsx")
dwa_ref = pd.read_excel("data/demo/TaskMatch/DWA Reference.xlsx")
dwa_ref['Work Activities Element ID'] = dwa_ref['Element ID']
skills = pd.read_excel("data/demo/TaskMatch/Skills to Work Activities.xlsx")
abilities = pd.read_excel("data/demo/TaskMatch/Abilities to Work Activities.xlsx")

## Setup

### Merge Task - DWA Map to Skills and Abilities Crosswalk

In [19]:
dwa_ref = dwa_ref.merge(skills, how='left', on = 'Work Activities Element ID')
dwa_ref = dwa_ref.merge(abilities, how='left', on = 'Work Activities Element ID')

activities = dwas.merge(dwa_ref, how = 'left', on = 'DWA ID')

###  Explode Tasks  - 1 row per Task

In [20]:
data['TaskMatch'][0]

"[('1117', 'Prepare required paperwork pertaining to departmental functions.'), ('16778', 'Confer with departments, such as marketing, business development, or operations, to coordinate product development or improvement.')]"

In [21]:
data['TaskMatch'] = data['TaskMatch'].apply(ast.literal_eval)
data['TaskMatch'][0]

[('1117', 'Prepare required paperwork pertaining to departmental functions.'),
 ('16778',
  'Confer with departments, such as marketing, business development, or operations, to coordinate product development or improvement.')]

In [22]:
tasks = data.explode('TaskMatch')
tasks

,id,onet_id,job_title,company,job_location,salary,text,responsibilities_text,requirements_text,preferred_text,company_desc,TitleMatch,TM_onet_id,FirmExtract,TaskMatch,ed_inferred_rule,ed_inferred_label,ed_inferred_confidence
0,1,43-4131.00,SBA Loan Associate,Bankers Healthcare Group,"Fort Lauderdale, FL",NaN,assist business development officer with sba 7...,Assist Business Development Officer with SBA 7...,A successful candidate is a motivated administ...,NaN,Are you ready to join a growing team that puts...,"('Loan Assistant', '43-4131.00', 0.903)",43-4131.00,"{'ex solutions', '-', 'fund'}","(1117, Prepare required paperwork pertaining t...",NaN,0.00000,NaN
0,1,43-4131.00,SBA Loan Associate,Bankers Healthcare Group,"Fort Lauderdale, FL",NaN,assist business development officer with sba 7...,Assist Business Development Officer with SBA 7...,A successful candidate is a motivated administ...,NaN,Are you ready to join a growing team that puts...,"('Loan Assistant', '43-4131.00', 0.903)",43-4131.00,"{'ex solutions', '-', 'fund'}","(16778, Confer with departments, such as marke...",NaN,0.00000,NaN
1,2,43-4131.00,Credit Analyst,First American Equipment Finance,"Rochester, NY",NaN,the credit analyst is responsible for assistin...,The Credit Analyst is responsible for assistin...,"Bachelor’s degree preferred, or equivalent com...",NaN,"First American Equipment Finance, an RBC/City ...","('Credit Analyst', '13-2041.00', 1.0)",13-2041.00,"{'american', 'first american'}",NaN,NaN,0.00000,NaN
2,3,19-1042.00,"Scientist, Lead Discovery","Accent Therapeutics, Inc.","Lexington, MA",NaN,design and execution of biochemical assays for...,Design and execution of biochemical assays for...,"BA/BS in Biochemistry, Enzymology or related f...",NaN,"As a key member of the Lead Discovery team, re...","('Lead Miner', '47-5049.00', 0.895)",47-5049.00,{'accent'},"(1574, Operate and maintain laboratory equipme...",NaN,NaN,NaN
2,3,19-1042.00,"Scientist, Lead Discovery","Accent Therapeutics, Inc.","Lexington, MA",NaN,design and execution of biochemical assays for...,Design and execution of biochemical assays for...,"BA/BS in Biochemistry, Enzymology or related f...",NaN,"As a key member of the Lead Discovery team, re...","('Lead Miner', '47-5049.00', 0.895)",47-5049.00,{'accent'},"(6109, Supervise laboratory sessions.)",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,987,31-1121.00,Home Health Aide - Home Health - New River Val...,Carilion Clinic,"Radford, VA",NaN,this position supports carilion's hallmarks of...,This position supports Carilion's hallmarks of...,"Education: Demonstrate ability to read, write ...",NaN,This is Carilion Clinic ...\n\nAn organization...,"('Home Health Attendant', '31-1121.00', 0.92)",31-1121.00,{'carilion clinic'},"(21127, Teach writing or communication classes.)",NaN,0.00000,NaN
986,987,31-1121.00,Home Health Aide - Home Health - New River Val...,Carilion Clinic,"Radford, VA",NaN,this position supports carilion's hallmarks of...,This position supports Carilion's hallmarks of...,"Education: Demonstrate ability to read, write ...",NaN,This is Carilion Clinic ...\n\nAn organization...,"('Home Health Attendant', '31-1121.00', 0.92)",31-1121.00,{'carilion clinic'},"(2117, Maintain records of prisoners' identifi...",NaN,0.00000,NaN
987,988,31-1121.00,Home Health Aide,Ridgeview Medical Center,"Waconia, MN",NaN,documents the various aspects of care on appro...,Documents the various aspects of care on appro...,Minimum Education/Work Experience \nOn Minneso...,"Preferred Qualifications: Previous home care, ...",NaN,"('Home Health Aides', '31-1121.00', 0.987)",31-1121.00,{'ridgeview'},"(18855, Conduct reference or background checks...",NaN,0.00000,NaN
988,989,31-1121.00,Home Health Aide,St. Claire HealthCare,"Morehead, KY",NaN,"observes, reports and documents patient status...","Observes, reports and documents patient status...","Skills/Abilities: Basic ability to read, write...",NaN,NaN,"('Home He

### Merge Tasks with DWA Reference

In [23]:
tasks[['Task ID', 'Task Description']] = pd.DataFrame(tasks['TaskMatch'].tolist(), index=tasks.index)
tasks['Task ID'] = tasks['Task ID'].fillna(0).astype('int64')
merged = tasks.merge(activities, how = 'left', on="Task ID")
merged

,id,onet_id,job_title,company,job_location,salary,text,responsibilities_text,requirements_text,preferred_text,...,IWA ID,IWA Title,DWA Title_y,Work Activities Element ID,Skills Element ID,Skills Element Name,Work Activities Element Name_x,Abilities Element ID,Abilities Element Name,Work Activities Element Name_y
0,1,43-4131.00,SBA Loan Associate,Bankers Healthcare Group,"Fort Lauderdale, FL",NaN,assist business development officer with sba 7...,Assist Business Development Officer with SBA 7...,A successful candidate is a motivated administ...,NaN,...,4.A.3.b.6.I08,Maintain operational records.,Document organizational or operational procedu...,4.A.3.b.6,2.A.1.c,Writing,Documenting/Recording Information,1.A.1.a.2,Written Comprehension,Documenting/Recording Information
1,1,43-4131.00,SBA Loan Associate,Bankers Healthcare Group,"Fort Lauderdale, FL",NaN,assist business development officer with sba 7...,Assist Business Development Officer with SBA 7...,A successful candidate is a motivated administ...,NaN,...,4.A.3.b.6.I08,Maintain operational records.,Document organizational or operational procedu...,4.A.3.b.6,2.A.1.c,Writing,Documenting/Recording Information,1.A.1.a.4,Written Expression,Documenting/Recording Information
2,1,43-4131.00,SBA Loan Associate,Bankers Healthcare Group,"Fort Lauderdale, FL",NaN,assist business development officer with sba 7...,Assist Business Development Officer with SBA 7...,A successful candidate is a motivated administ...,NaN,...,4.A.3.b.6.I08,Maintain operational records.,Document organizational or operational procedu...,4.A.3.b.6,2.A.1.c,Writing,Documenting/Recording Information,1.A.1.b.6,Information Ordering,Documenting/Recording Information
3,1,43-4131.00,SBA Loan Associate,Bankers Healthcare Group,"Fort Lauderdale, FL",NaN,assist business development officer with sba 7...,Assist Business Development Officer with SBA 7...,A successful candidate is a motivated administ...,NaN,...,4.A.3.b.6.I08,Maintain operational records.,Document organizational or operational procedu...,4.A.3.b.6,2.A.1.c,Writing,Documenting/Recording Information,1.A.1.b.7,Category Flexibility,Documenting/Recording Information
4,1,43-4131.00,SBA Loan Associate,Bankers Healthcare Group,"Fort Lauderdale, FL",NaN,assist business development officer with sba 7...,Assist Business Development Officer with SBA 7...,A successful candidate is a motivated administ...,NaN,...,4.A.3.b.6.I08,Maintain operational records.,Document organizational or operational procedu...,4.A.3.b.6,2.A.1.c,Writing,Documenting/Recording Information,1.A.1.g.1,Selective Attention,Documenting/Recording Information
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208041,989,31-1121.00,Home Health Aide,St. Claire HealthCare,"Morehead, KY",NaN,"observes, reports and documents patient status...","Observes, reports and documents patient status...","Skills/Abilities: Basic ability to read, write...",NaN,...,4.A.4.b.5.I01,Coach others.,Coach others.,4.A.4.b.5,2.B.5.d,Management of Personnel Resources,Coaching and Developing Others,1.A.1.b.4,Deductive Reasoning,Coaching and Developing Others
208042,989,31-1121.00,Home Health Aide,St. Claire HealthCare,"Morehead, KY",NaN,"observes, reports and documents patient status...","Observes, reports and documents patient status...","Skills/Abilities: Basic ability to read, write...",NaN,...,4.A.4.b.5.I01,Coach others.,Coach others.,4.A.4.b.5,2.B.5.d,Management of Personnel Resources,Coaching and Developing Others,1.A.1.b.5,Inductive Reasoning,Coaching and Developing Others
208043,989,31-1121.00,Home Health Aide,St. Claire HealthCare,"Morehead, KY",NaN,"observes, reports and documents patient status...","Observes, reports and documents patient status...","Skills/Abilities: Basic ability to read, write...",NaN,...,4.A.4.b.5.I01,Coach others.,Coach others.,4.A.4.b.5,2.B.5.d,Management of Personnel Resources,Coaching and Developing Others,1.A.4.b.4,Speech Recognition,Coaching and Developing Others
2

## Create Pivot Tables

In [24]:
merged['one'] = 1
merged['onet_short'] = merged['onet_id'].str[:2]

occ_table = pd.pivot_table(merged, values='one', index=['Element Name'],
                       columns=['onet_short'], aggfunc="sum")
occ_table = occ_table.fillna(0)

skill_table = pd.pivot_table(merged, values='one', index=['Element Name'],
                       columns=['Skills Element Name'], aggfunc="sum")
skill_table = skill_table.fillna(0)

abilities_table = pd.pivot_table(merged, values='one', index=['Element Name'],
                       columns=['Abilities Element Name'], aggfunc="sum")
abilities_table = abilities_table.fillna(0)

skills_occ_table = pd.pivot_table(merged, values='one', index=['onet_short'],
                       columns=['Skills Element Name'], aggfunc="sum")
skills_occ_table = skills_occ_table.fillna(0)

abilities_occ_table = pd.pivot_table(merged, values='one', index=['onet_short'],
                       columns=['Abilities Element Name'], aggfunc="sum")
abilities_occ_table = abilities_occ_table.fillna(0)
abilities_occ_table

Abilities Element Name,Arm-Hand Steadiness,Auditory Attention,Category Flexibility,Control Precision,Deductive Reasoning,Depth Perception,Dynamic Strength,Explosive Strength,Extent Flexibility,Far Vision,...,Speed of Closure,Stamina,Static Strength,Time Sharing,Trunk Strength,Visual Color Discrimination,Visualization,Wrist-Finger Speed,Written Comprehension,Written Expression
onet_short,,,,,,,,,,,,,,,,,,,,,
11,0.0,49.0,849.0,0.0,1446.0,4.0,2.0,2.0,2.0,82.0,...,53.0,2.0,2.0,45.0,2.0,52.0,21.0,9.0,1438.0,1299.0
13,0.0,10.0,147.0,0.0,347.0,5.0,0.0,0.0,0.0,21.0,...,0.0,0.0,0.0,10.0,0.0,10.0,9.0,9.0,319.0,275.0
15,214.0,300.0,3317.0,210.0,5393.0,155.0,3.0,7.0,199.0,380.0,...,140.0,3.0,7.0,160.0,3.0,269.0,660.0,75.0,4950.0,4083.0
17,20.0,46.0,353.0,18.0,707.0,26.0,0.0,2.0,18.0,51.0,...,0.0,0.0,2.0,30.0,0.0,41.0,56.0,9.0,611.0,539.0
19,21.0,29.0,292.0,20.0,598.0,24.0,1.0,2.0,18.0,47.0,...,10.0,1.0,2.0,5.0,1.0,29.0,54.0,9.0,549.0,516.0
21,2.0,15.0,900.0,2.0,1427.0,0.0,2.0,2.0,2.0,15.0,...,28.0,2.0,2.0,15.0,2.0,15.0,14.0,10.0,1344.0,1346.0
25,25.0,5.0,423.0,18.0,667.0,0.0,1.0,8.0,24.0,17.0,...,0.0,1.0,8.0,5.0,1.0,5.0,18.0,6.0,648.0,556.0
29,69.0,157.0,963.0,56.0,1920.0,72.0,9.0,22.0,70.0,187.0,...,31.0,9.0,22.0,85.0,9.0,157.0,128.0,25.0,1696.0,1395.0
31,4.0,13.0,215.0,2.0,416.0,8.0,5.0,7.0,7.0,13.0,...,7.0,5.0,7.0,5.0,5.0,13.0,18.0,7.0,339.0,338.0


## Take Natural Log of Non-Normal Counts

In [25]:
skills_occ_table = skills_occ_table.apply(lambda x: np.log2(1 + x))
abilities_occ_table = abilities_occ_table.apply(lambda x: np.log2(1 + x))
abilities_occ_table

Abilities Element Name,Arm-Hand Steadiness,Auditory Attention,Category Flexibility,Control Precision,Deductive Reasoning,Depth Perception,Dynamic Strength,Explosive Strength,Extent Flexibility,Far Vision,...,Speed of Closure,Stamina,Static Strength,Time Sharing,Trunk Strength,Visual Color Discrimination,Visualization,Wrist-Finger Speed,Written Comprehension,Written Expression
onet_short,,,,,,,,,,,,,,,,,,,,,
11,0.000000,5.643856,9.731319,0.000000,10.498849,2.321928,1.584963,1.584963,1.584963,6.375039,...,5.754888,1.584963,1.584963,5.523562,1.584963,5.727920,4.459432,3.321928,10.490851,10.344296
13,0.000000,3.459432,7.209453,0.000000,8.442943,2.584963,0.000000,0.000000,0.000000,4.459432,...,0.000000,0.000000,0.000000,3.459432,0.000000,3.459432,3.321928,3.321928,8.321928,8.108524
15,7.748193,8.233620,11.696098,7.721099,12.397140,7.285402,2.000000,3.000000,7.643856,8.573647,...,7.139551,2.000000,3.000000,7.330917,2.000000,8.076816,9.368506,6.247928,12.273504,11.995767
17,4.392317,5.554589,8.467606,4.247928,9.467606,4.754888,0.000000,1.584963,4.247928,5.700440,...,0.000000,0.000000,1.584963,4.954196,0.000000,5.392317,5.832890,3.321928,9.257388,9.076816
19,4.459432,4.906891,8.194757,4.392317,9.226412,4.643856,1.000000,1.584963,4.247928,5.584963,...,3.459432,1.000000,1.584963,2.584963,1.000000,4.906891,5.781360,3.321928,9.103288,9.014020
21,1.584963,4.000000,9.815383,1.584963,10.479780,0.000000,1.584963,1.584963,1.584963,4.000000,...,4.857981,1.584963,1.584963,4.000000,1.584963,4.000000,3.906891,3.459432,10.393390,10.395534
25,4.700440,2.584963,8.727920,4.247928,9.383704,0.000000,1.000000,3.169925,4.643856,4.169925,...,0.000000,1.000000,3.169925,2.584963,1.000000,2.584963,4.247928,2.807355,9.342075,9.121534
29,6.129283,7.303781,9.912889,5.832890,10.907642,6.189825,3.321928,4.523562,6.149747,7.554589,...,5.000000,3.321928,4.523562,6.426265,3.321928,7.303781,7.011227,4.700440,10.728771,10.447083
31,2.321928,3.807355,7.754888,1.584963,8.703904,3.169925,2.584963,3.000000,3.000000,3.807355,...,3.000000,2.584963,3.000000,2.584963,2.584963,3.807355,4.247928,3.000000,8.409391,8.405141


## Build scales for Importance 

In [26]:
# see: https://www.onetonline.org/help/online/scales
#This rating indicates the degree of importance a particular descriptor is to the occupation. The possible ratings range from "Not Important" (1) to "Extremely Important" (5).
#importance of the abilitiy relative to other abilities to the occupation

# Proportion of x within an occupation
abilities_occ_importance = abilities_occ_table.apply(lambda x: x/abilities_occ_table.max(axis=1))
# Turn 0s to NaN
abilities_occ_importance = abilities_occ_importance.replace(0, None)

# Standardize and center at 2.5
abilities_occ_importance = abilities_occ_importance.apply(lambda x: 2.5 + (x- abilities_occ_importance.mean(axis=1)) / abilities_occ_importance.std(axis=1))

# Restore 0s and Round to 2 digits
abilities_occ_importance = abilities_occ_importance.fillna(0)
abilities_occ_importance = abilities_occ_importance.round(2)

skills_occ_importance = skills_occ_table.apply(lambda x: x/skills_occ_table.max(axis=1))
skills_occ_importance = skills_occ_importance.replace(0, None)
skills_occ_importance = skills_occ_importance.apply(lambda x: 2.5 + (x- skills_occ_importance.mean(axis=1)) / skills_occ_importance.std(axis=1))
skills_occ_importance = skills_occ_importance.fillna(0)
skills_occ_importance = skills_occ_importance.round(1)
skills_occ_importance


Skills Element Name,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,Instructing,Judgment and Decision Making,Learning Strategies,...,Repairing,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
onet_short,,,,,,,,,,,,,,,,,,,,,
11,3.0,3.4,2.4,3.1,3.4,0.0,0.0,3.1,2.9,2.9,...,0.0,3.1,3.2,3.3,2.3,2.4,1.2,2.7,0.5,3.2
13,2.8,3.4,2.7,3.0,3.8,0.0,0.0,3.0,2.7,2.7,...,0.0,3.0,3.0,3.3,2.3,2.5,0.2,2.7,0.0,3.5
15,2.9,3.5,3.1,3.2,3.8,1.6,1.6,3.1,3.0,2.8,...,1.6,3.2,3.3,3.4,2.9,2.8,2.4,2.6,2.2,3.5
17,3.1,3.6,2.9,3.2,3.9,1.1,1.1,3.2,3.0,3.0,...,1.1,3.2,3.3,3.5,2.4,2.7,1.4,2.9,1.9,3.5
19,3.1,3.5,2.4,3.3,3.7,1.0,1.0,3.3,2.7,3.1,...,1.0,3.3,3.3,3.4,2.1,2.2,1.4,2.7,2.1,3.4
21,2.8,3.2,2.6,3.2,3.2,0.0,0.0,3.1,2.9,2.8,...,0.0,3.1,3.1,3.2,2.6,2.6,1.1,2.4,0.0,3.2
25,3.3,3.5,2.7,3.4,3.6,1.5,1.5,3.4,3.3,3.3,...,1.5,3.4,3.4,3.4,2.3,2.4,0.6,2.7,1.5,3.5
29,3.1,3.6,2.5,3.3,3.7,1.5,1.5,3.2,3.0,3.1,...,1.5,3.3,3.4,3.5,2.0,2.1,0.7,2.6,2.4,3.4
31,3.1,3.6,2.5,3.4,3.6,0.0,0.0,3.2,3.0,3.1,...,0.0,3.3,3.4,3.6,2.3,2.4,1.7,2.8,1.6,3.6


## Build Scales for Level

In [27]:
#Level
# level of the ability relative to level required qin other occupations
# This rating indicates the degree, or point along a continuum, to which a particular descriptor is required or needed to perform the occupation.

# Proportion of x within a skill or ability
abilities_occ_level = abilities_occ_table.apply(lambda x: x/x.max())
abilities_occ_level = abilities_occ_level.replace(0, None)
abilities_occ_level = abilities_occ_level.apply(lambda x: 2.5 + (x- x.mean()) / x.std())
abilities_occ_level = abilities_occ_level.fillna(0)
abilities_occ_level = abilities_occ_level.round(1)

skills_occ_level = skills_occ_table.apply(lambda x: x/x.max())
skills_occ_level = skills_occ_level.replace(0, None)
skills_occ_level = skills_occ_level.apply(lambda x: 2.5 + (x- x.mean()) / x.std())
skills_occ_level = skills_occ_level.fillna(0)
skills_occ_level = skills_occ_level.round(1)
skills_occ_level

Skills Element Name,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,Instructing,Judgment and Decision Making,Learning Strategies,...,Repairing,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
onet_short,,,,,,,,,,,,,,,,,,,,,
11,3.2,3.1,2.5,2.9,2.9,0.0,0.0,3.0,3.0,3.1,...,0.0,3.0,3.1,3.1,2.8,2.8,2.5,3.2,1.1,2.9
13,0.8,1.0,1.5,0.9,1.2,0.0,0.0,0.9,1.0,0.8,...,0.0,0.9,0.9,1.0,1.7,1.7,1.4,1.4,0.0,1.1
15,4.4,4.5,4.6,4.4,4.6,3.5,3.5,4.5,4.6,4.3,...,3.5,4.4,4.4,4.5,5.0,4.9,5.1,4.6,3.8,4.7
17,2.0,2.0,2.2,1.8,2.1,1.8,1.8,1.9,1.9,1.9,...,1.8,1.8,1.8,2.0,2.2,2.4,2.5,2.2,2.3,2.0
19,1.9,1.8,1.6,2.0,1.9,1.8,1.8,2.0,1.6,2.1,...,1.8,2.0,2.0,1.9,1.8,1.7,2.5,2.0,2.4,1.9
21,2.9,2.9,2.9,3.3,2.6,0.0,0.0,3.3,3.2,3.1,...,0.0,3.3,3.2,3.1,3.5,3.4,2.5,2.7,0.0,3.1
25,2.2,1.9,1.8,2.2,1.8,1.8,1.8,2.3,2.3,2.3,...,1.8,2.2,2.1,2.0,1.7,1.7,0.9,1.7,1.5,2.0
29,3.3,3.4,2.7,3.4,3.2,2.5,2.5,3.3,3.2,3.5,...,2.5,3.4,3.4,3.4,2.4,2.5,2.0,3.0,3.2,3.3
31,1.3,1.4,1.2,1.5,1.2,0.0,0.0,1.3,1.3,1.4,...,0.0,1.5,1.5,1.6,1.5,1.2,2.2,1.4,1.5,1.5


In [28]:
skills_occ_level_std = skills_occ_table.apply(lambda x:  ((( x/x.max()) * (1-( x/x.max()))) / x.sum()) ** .5)
skills_occ_level_std

Skills Element Name,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,Instructing,Judgment and Decision Making,Learning Strategies,...,Repairing,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
onet_short,,,,,,,,,,,,,,,,,,,,,
11,0.031087,0.031502,0.044135,0.033700,0.033651,0.000000,0.000000,0.033577,0.035314,0.031232,...,0.000000,0.033338,0.031715,0.032607,0.046839,0.043896,0.061203,0.034498,0.058469,0.034067
13,0.045743,0.043514,0.048402,0.045095,0.042481,0.000000,0.000000,0.045291,0.046709,0.046198,...,0.000000,0.045128,0.045066,0.044320,0.051481,0.049767,0.059620,0.046581,0.000000,0.043323
15,0.000000,0.000000,0.000000,0.000000,0.000000,0.050882,0.050882,0.000000,0.000000,0.000000,...,0.050882,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030232,0.000000
17,0.040804,0.039387,0.045595,0.041703,0.038792,0.070968,0.070968,0.041115,0.043151,0.041240,...,0.070968,0.041655,0.041235,0.040308,0.050005,0.046729,0.061203,0.042007,0.058942,0.039732
19,0.040990,0.040024,0.048206,0.040527,0.039643,0.070968,0.070968,0.040567,0.044772,0.039952,...,0.070968,0.040499,0.040613,0.040673,0.051176,0.049767,0.061203,0.043683,0.057598,0.040447
21,0.034415,0.032588,0.040696,0.029850,0.035779,0.000000,0.000000,0.029900,0.033530,0.031566,...,0.000000,0.030166,0.030695,0.032000,0.040814,0.038628,0.061203,0.038394,0.000000,0.032535
25,0.039129,0.039600,0.047493,0.039039,0.040080,0.070968,0.070968,0.039151,0.040783,0.038384,...,0.070968,0.039513,0.039663,0.040284,0.051511,0.049803,0.055581,0.044999,0.060362,0.039780
29,0.029751,0.027557,0.042285,0.028712,0.030959,0.067927,0.067927,0.029940,0.033778,0.027101,...,0.067927,0.028882,0.028476,0.029146,0.048811,0.046297,0.061438,0.036007,0.047767,0.030335
31,0.044163,0.042076,0.049273,0.042934,0.042472,0.000000,0.000000,0.043964,0.045739,0.043732,...,0.000000,0.043141,0.042980,0.042060,0.052070,0.051010,0.061532,0.046412,0.060362,0.041945


In [29]:
skills_occ_importance_std = skills_occ_table.apply(lambda x: ((( x/skills_occ_table.max(axis=1)) * (1-( x/skills_occ_table.max(axis=1)))) / skills_occ_table.sum(axis=1)) ** .5)
skills_occ_importance_std

Skills Element Name,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,Instructing,Judgment and Decision Making,Learning Strategies,...,Repairing,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
onet_short,,,,,,,,,,,,,,,,,,,,,
11,0.019413,0.006879,0.028107,0.018337,0.000000,0.000000,0.000000,0.019117,0.021651,0.021461,...,0.000000,0.017679,0.014913,0.012414,0.029510,0.028596,0.033760,0.025237,0.032348,0.015012
13,0.027865,0.018262,0.029415,0.025362,0.000000,0.000000,0.000000,0.025926,0.028597,0.029497,...,0.000000,0.025362,0.025362,0.021527,0.032597,0.030905,0.037951,0.029005,0.000000,0.016039
15,0.019358,0.011537,0.017882,0.016593,0.000000,0.026574,0.026574,0.017410,0.018522,0.020592,...,0.026574,0.016441,0.015974,0.013317,0.019708,0.020701,0.023483,0.022139,0.024136,0.013091
17,0.022329,0.013673,0.025449,0.021853,0.000000,0.033232,0.033232,0.021222,0.024160,0.023997,...,0.033232,0.021632,0.020687,0.016872,0.029288,0.027097,0.032993,0.025349,0.031683,0.015312
19,0.021753,0.013166,0.029594,0.018252,0.000000,0.034253,0.034253,0.018984,0.026301,0.021639,...,0.034253,0.017980,0.017980,0.015912,0.031208,0.030752,0.033886,0.026742,0.031116,0.015209
21,0.020956,0.000000,0.024188,0.008853,0.007068,0.000000,0.000000,0.011339,0.018194,0.020286,...,0.000000,0.009087,0.009087,0.004521,0.024389,0.024389,0.034489,0.026965,0.000000,0.006662
25,0.017165,0.008486,0.027627,0.012485,0.000000,0.033931,0.033931,0.014000,0.018094,0.018010,...,0.033931,0.013667,0.013667,0.012637,0.031177,0.030209,0.031107,0.027362,0.033931,0.010432
29,0.019343,0.004524,0.025321,0.015034,0.000000,0.030385,0.030385,0.017093,0.020738,0.019539,...,0.030385,0.014969,0.013969,0.010994,0.028730,0.028150,0.030285,0.025038,0.026603,0.012491
31,0.022232,0.005260,0.030930,0.015570,0.003710,0.000000,0.000000,0.020164,0.024414,0.022308,...,0.000000,0.016266,0.015392,0.003710,0.032709,0.032301,0.035901,0.027353,0.036120,0.000000


In [30]:
# Plot the Skills Level for first 3 Major Occupation Groups

In [31]:
import plotly.express as px
import plotly.graph_objects as go

figy = list(skills_occ_level.columns.values)
figx = skills_occ_level.iloc[0, 0:]
figx = figx.tolist()

figx2 = skills_occ_level.iloc[1, 0:]
figx2 = figx2.tolist()

figx3 = skills_occ_level.iloc[2, 0:]
figx3 = figx3.tolist()

fig = go.Figure()
fig.add_trace(go.Bar(
    y= figy,
    x= figx,
    name='Management Occupations',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y= figy,
    x= figx2,
    name='Business and Finance Occupations',
    orientation='h',
    marker=dict(
        color='LightSkyBlue', opacity = .4,
        line=dict(color='LightSkyBlue', width=3)
    )
))
fig.add_trace(go.Bar(
    y= figy,
    x= figx3,
    name='Computer and Mathematical Occupations',
    orientation='h',
    marker=dict(
        color='MediumPurple', opacity = .4,
        line=dict(color='MediumPurple', width=3)
    )
))

fig.update_layout(barmode='overlay')
fig.show()

## Plot the Skills Importance for 3 Major Occupational Groups

In [32]:
import plotly.express as px
import plotly.graph_objects as go

figy = list(skills_occ_importance.columns.values)
figx = skills_occ_importance.iloc[0, 0:]
figx = figx.tolist()

figx2 = skills_occ_importance.iloc[1, 0:]
figx2 = figx2.tolist()

figx3 = skills_occ_importance.iloc[2, 0:]
figx3 = figx3.tolist()

fig = go.Figure()
fig.add_trace(go.Bar(
    y= figy,
    x= figx,
    name='Management Occupations',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y= figy,
    x= figx2,
    name='Business and Finance Occupations',
    orientation='h',
    marker=dict(
        color='LightSkyBlue', opacity = .4,
        line=dict(color='LightSkyBlue', width=3)
    )
))
fig.add_trace(go.Bar(
    y= figy,
    x= figx3,
    name='Computer and Mathematical Occupations',
    orientation='h',
    marker=dict(
        color='MediumPurple', opacity = .4,
        line=dict(color='MediumPurple', width=3)
    )
))

fig.update_layout(barmode='overlay')
fig.show()